|  |  |
| ---: | :--- |
| Выполнил:| Волонцевич Олег |
| Дата: | 29.08.2022 |


# Парсинг диалогов из файла .csv

Задачи:

* a. Извлекать реплики с приветствием - где менеджер поздоровался.
* b. Извлекать реплики, где менеджер представил себя. 
* c. Извлекать имя менеджера.
* d. Извлекать название компании.
* e. Извлекать реплики, где менеджер попрощался.
* f. Проверять требование к менеджеру: "в каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом"

In [1]:
#pip install yargy

In [2]:
import pandas as pd

import nltk
import pymorphy2

from yargy import Parser, rule
from yargy.predicates import gram, dictionary

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
data = pd.read_csv('test_data.csv')
data

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкретно продам
476,5,139,manager,Все я вам высылаю счет и с вами на связи если будут вопросы можете писать на ватсапе
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


Файл содержит преобразованные в текст телефонные звонки. Отсутствуют знаки препинания, не всегда есть разбивка на предложения, отсутствует верхний регистр кроме первого символа строки (в т.ч. в именах).

### Приветствие
* a. Извлекать реплики с приветствием - где менеджер поздоровался.

In [5]:
def greeting(row):       
    '''
    функция возвращает True если менеджер поздоровался
    '''
    _txt = row['text'].lower()
    
    if row['role'] != 'manager':
        return False
    else:
        for _gree in greeting_dict:
            if _gree in _txt:
                return True
                break
        return False

При необходимости можно дополнительно использовать условие для поиска приветствия только в N первых строчках диалога:

* if row['line_n'] >=N:
    *     return False  

In [6]:
greeting_dict = ['добрый день', 'доброе утро', 'добрый вечер', 'здравствуйте', 'приветствую', 'привет']#словарь приветствий
data['greeting'] = data.apply(greeting, axis=1)
data.head(5)

,dlg_id,line_n,role,text,greeting
0,0,0,client,Алло,False
1,0,1,manager,Алло здравствуйте,True
2,0,2,client,Добрый день,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,False
4,0,4,client,Ага,False


### Представился
* b. Извлекать реплики, где менеджер представил себя. 

In [7]:
def introduce(row):       
    '''
    функция возвращает True если менеджер представился 
    (если в пяти первых строчках диалога использованы обороты из introduce_dict)
    '''
    _txt = row['text'].lower()
    
    if row['line_n'] >5:
        return False 
    elif row['role'] != 'manager':
        return False
    else:
        for _intr in introduce_dict:
            if _intr in _txt:
                return True
                break
        return False

In [8]:
introduce_dict = ['зовут', 'представиться', 'беспокоит', 'это', 'мое имя']           
data['introduce'] = data.apply(introduce, axis=1)
data[data['dlg_id']==3].head(5)

,dlg_id,line_n,role,text,greeting,introduce
249,3,0,client,Добрый день,False,False
250,3,1,manager,Алло дмитрий добрый день,True,False
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,False,True
252,3,3,client,Да удобно,False,False
253,3,4,manager,Да дмитрий вот мне моя коллега анастасия подсказала что у вас есть какие то открытые вопросы связанные с техническими особенностями а мы серым вот готов буду его ответить или взять паузу и решите эти вопросы уточните пожалуйста в чем задача состоит,False,False


### ФИО
* c. Извлекать имя менеджера.

In [9]:
def manager_name(row):       
    '''
    функция ищет ФИО среди первых пяти строчек диалога в репликах где менеджер представился
    Если есть Name-сущности - возвращает собранную из них строку иначе None
    '''
    _txt = row['text']
    _name = None
    if row['line_n'] >5:
        return None 
    elif row['role'] != 'manager':
        return None
    elif row['introduce'] != True:
        return None
    else:
        for word in nltk.word_tokenize(_txt):
            for p in morph.parse(word):
                if 'Name' in p.tag and p.score >= prob_thresh:
                    _name = '' if _name == None else _name
                    _name += word
                    _name += ' '
        _name = None if _name == None else _name.title().rstrip()
        return _name

In [10]:
prob_thresh = 0.4
morph = pymorphy2.MorphAnalyzer()
data['manager'] = data.apply(manager_name, axis=1)
data[data['manager']==data['manager']]

,dlg_id,line_n,role,text,greeting,introduce,manager
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,False,True,Ангелина
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,False,True,Ангелина
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,False,True,Ангелина
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,False,True,Максим
338,5,1,manager,Да это анастасия,False,True,Анастасия


### Компания
* d. Извлекать название компании.

In [11]:
def company_name(row):       
    '''
    функция ищет названия компаний согласно заданному правилу для Yargy-парсера
    '''
    _txt = row['text'].lower()
    _company_name = ''
    for match in parser.findall(_txt):
        for x in match.tokens:
            _company_name += x.value
            _company_name += ' '
        return _company_name.rstrip()

In [12]:
company_rule = rule (dictionary({'компания', 'фирма', 'ооо', 'зао', 'оао', 'ао', 'пао', 'нко', 'оп', 'группа компаний',
                                 'торговый дом', 'ип'}),
                     gram('ADVB').optional(), gram('ADJF').optional(), gram('NOUN').optional(), 
                     dictionary({'и', 'энд'}).optional(), gram('NOUN'))

parser = Parser(company_rule)
data['company'] = data.apply(company_name, axis=1)

Правило позволяет находить сложные и "заковыристые" названия.
Тестировалась на списке:
* компания китобизнес 
* фирма диджитал бизнес 
* группа компаний крутой замес 
* компания рога и копыта 
* ооо вкусно и точка 
* торговый дом матвеев и сыновья 
* ип николаев игорь юрьевич

In [13]:
data[data['company']==data['company']]

,dlg_id,line_n,role,text,greeting,introduce,manager,company
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,False,True,Ангелина,компания диджитал бизнес
28,0,28,client,А вот компания в которой я работаю,False,False,None,компания в
50,0,50,client,Это хорошая компания и 110,False,False,None,компания и
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,False,True,Ангелина,компания диджитал бизнес
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,False,True,Ангелина,компания диджитал бизнес
251,3,2,manager,Добрый меня максим зовут компания китобизнес удобно говорить,False,True,Максим,компания китобизнес


Есть 2 ложных срабатывания правила (строки 28 и 50). Можно исключить костылем (например по количеству символов) или усложнением логики правила.

### Прощание
* e. Извлекать реплики, где менеджер попрощался.

In [14]:
def goodbye(row):       
    '''
    функция возвращает True если менеджер попрощался
    '''
    _txt = row['text'].lower()
    
    if row['role'] != 'manager':
        return False
    else:
        for _bye in goodbye_dict:
            if _bye in _txt:
                return True
                break
        return False

Так же как и с приветствием при необходимости можно искать прощание только в последних строчках диалога.

In [15]:
goodbye_dict = ['до свидания', 'до связи', 'до звонка', 'до встречи', 'до скорого', 'пока', 'счастливо', 
                'всего хорошего', 'всего вам хорошего', 'всего доброго', 'всего вам доброго', 
                'удачного дня', 'удачного вам дня', 'хорошего дня', 'успехов']
data['goodbye'] = data.apply(goodbye, axis=1)
data[data['dlg_id']==3].tail(5)

,dlg_id,line_n,role,text,greeting,introduce,manager,company,goodbye
297,3,48,client,Давайте часов в 11 наверное,False,False,None,None,False
298,3,49,manager,11 да смогу набрать в 11 наберу вас,False,False,None,None,False
299,3,50,client,Угу все хорошо,False,False,None,None,False
300,3,51,manager,Угу все хорошо да понедельника тогда всего доброго,False,False,None,None,True
301,3,52,client,Да до свидания,False,False,None,None,False


### Приветствие + прощание
* f. Проверять требование к менеджеру: "в каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом"

In [16]:
hello_goodbye = data.pivot_table(index=data['dlg_id'], aggfunc={'greeting': (sum), 'goodbye': (sum)})
hello_goodbye['hello_goodbye'] = (hello_goodbye['greeting'] * hello_goodbye['goodbye']) > 0
hello_goodbye

,goodbye,greeting,hello_goodbye
dlg_id,,,
0,1,1,True
1,2,1,True
2,0,1,False
3,1,1,True
4,1,0,False
5,1,0,False


Для наглядности добавим менеджера диалога.

In [17]:
dlg_manager = data[data['manager']==data['manager']]
dlg_manager.set_index('dlg_id', inplace=True)
hello_goodbye = pd.merge(hello_goodbye, dlg_manager['manager'], how="left", on="dlg_id")
hello_goodbye.style.highlight_null(null_color='yellow')

,goodbye,greeting,hello_goodbye,manager
dlg_id,,,,
0,1,1,True,Ангелина
1,2,1,True,Ангелина
2,0,1,False,Ангелина
3,1,1,True,Максим
4,1,0,False,nan
5,1,0,False,Анастасия


### Результат

Получили таблицу c промаркированными True/False строками в которых менеджер поздоровался, представился, попрощался, а также извлеченными именами и названиями компании.

In [18]:
data[data['dlg_id']==1].head(5)

,dlg_id,line_n,role,text,greeting,introduce,manager,company,goodbye
109,1,0,client,Да здравствуйте когда заканчивается,False,False,None,None,False
110,1,1,manager,Алло здравствуйте,True,False,None,None,False
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,False,True,Ангелина,компания диджитал бизнес,False
112,1,3,manager,А так у вас заканчивается 25 февраля,False,False,None,None,False
113,1,4,manager,На полгода минимальный,False,False,None,None,False


Проверка требования "в каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом" выполнена отдельной таблицей.

In [19]:
hello_goodbye

,goodbye,greeting,hello_goodbye,manager
dlg_id,,,,
0,1,1,True,Ангелина
1,2,1,True,Ангелина
2,0,1,False,Ангелина
3,1,1,True,Максим
4,1,0,False,NaN
5,1,0,False,Анастасия


Результаты работы сохраняем в файлах csv

In [20]:
data.to_csv('pars_data.csv', encoding='cp1251')

In [21]:
hello_goodbye.to_csv('hello_goodbye_data.csv', encoding='cp1251')

Спасибо за интересную задачу и уделенное внимание.
В случае вашей заинтересованности с радостью рассмотрю варианты трудоустройства.
Волонцевич Олег
+7 (915) 254-31-96 тел. / WhatsApp / Telegram